In [1]:
import pandas as pd
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F

from torchvision import transforms

from random import randint

import os
from os import path

from functools import partial, reduce
from tqdm import tqdm

## Local Imports ##
from models import helpers as model_helpers, models as custom_models
from datasets import helpers as dataset_helpers, datasets as custom_datasets

In [2]:
W, H = (32, 32)
BS = 128
MAX_EPOCHS = 150

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DT_ROOT = 'data'
POLYGON_COUNT_DIR = path.join(DT_ROOT, 'polygon_data_counts')
POLYGON_PERCENTAGE_DIR = path.join(DT_ROOT, 'polygon_data_percentage')
ELLIPSE_COUNT_DIR = path.join(DT_ROOT, 'ellipse_data_counts')
ELLIPSE_PERCENTAGE_DIR = path.join(DT_ROOT, 'ellipse_data_percentage')
TRANSFORM = transforms.Compose([transforms.ToTensor()])

dataset_generators = [
    model_helpers.Param('RECT_COUNT', custom_datasets.get_dataset(
        root_dir=POLYGON_COUNT_DIR, 
        df_path=path.join(POLYGON_COUNT_DIR, 'data.csv'),
        transform=TRANSFORM,
        bs=BS
    )),
    model_helpers.Param('RECT_PCT', custom_datasets.get_dataset(
        root_dir=POLYGON_PERCENTAGE_DIR, 
        df_path=path.join(POLYGON_PERCENTAGE_DIR, 'data.csv'),
        transform=TRANSFORM,
        bs=BS
    )),
    model_helpers.Param('ELLIPSE_COUNT', custom_datasets.get_dataset(
        root_dir=ELLIPSE_COUNT_DIR, 
        df_path=path.join(ELLIPSE_COUNT_DIR, 'data.csv'),
        transform=TRANSFORM,
        bs=BS
    )),
    model_helpers.Param('ELLIPSE_PCT', custom_datasets.get_dataset(
        root_dir=ELLIPSE_PERCENTAGE_DIR, 
        df_path=path.join(ELLIPSE_PERCENTAGE_DIR, 'data.csv'),
        transform=TRANSFORM,
        bs=BS
    ))
]

models_to_test = [
    *custom_models.get_models(input_size=(1, W, H)), 
    model_helpers.Param('PERCEPTRON', lambda: nn.Sequential(model_helpers.Flatten(), nn.Linear(W * H, 1)))
]

print(f"Device: {DEVICE}")

Device: cuda


In [3]:
OPTIMS = [
    model_helpers.Param('Adam', lambda: torch.optim.Adam),
    model_helpers.Param('SGD', lambda: partial(torch.optim.SGD, lr=0.00001))
]

LOSS_FNS = [
    model_helpers.Param('L1LOSS', lambda: model_helpers.squeeze_loss(nn.L1Loss())),
    model_helpers.Param('MSELOSS', lambda: model_helpers.squeeze_loss(nn.MSELoss()))
]


grid = model_helpers.new_grid_search(models_to_test, OPTIMS, LOSS_FNS)
grid = list(grid)

print(f"Will train {len(LOSS_FNS) * len(models_to_test) * len(dataset_generators)} models")
print(f"{len(models_to_test)} MODELS")
print(f"{len(dataset_generators)} DATASETS")
print(f"{len(LOSS_FNS)} LOSS_FNS")
print(f"{len(OPTIMS)} OPTIMS")

Will train 40 models
5 MODELS
4 DATASETS
2 LOSS_FNS
2 OPTIMS


In [4]:
def normalize_train_test_df(df, indexes):
    train_losses = (pd.DataFrame(df.train_losses.tolist(), index=df.set_index(indexes).index).stack()
        .reset_index(name='train_losses'))
    val_losses = (pd.DataFrame(df.val_losses.tolist(), index=df.set_index(indexes).index).stack()
        .reset_index(name='val_losses'))
    epoch = (pd.DataFrame(df.epoch.tolist(), index=df.set_index(indexes).index).stack()
        .reset_index(name='epoch'))
    val_losses["epoch"] = epoch.epoch
    val_losses["train_losses"] = train_losses.train_losses
    return val_losses.drop(columns="level_4").set_index(indexes)

columns = [
        "model_name",
        "dataset",
        "optim",
        "loss_fn"
]

df = pd.DataFrame(
    columns=columns
)

for dt in dataset_generators:
    dl_train = dt.param.train()
    dl_test = dt.param.test()
    for row in grid:
        print(f"Pool: {row.model.name}\nOpt: {row.opt.name}\nLoss: {row.loss.name}\nDT: {dt.name}\n")
        metrics = model_helpers.train(
            dl_train, 
            dl_test, 
            row.opt.param(),
            row.loss.param(), 
            row.model.param(), 
            MAX_EPOCHS, 
            DEVICE)
        row = pd.Series({
            "model_name": row.model.name,
            "dataset": dt.name,
            "optim": row.opt.name,
            "loss_fn": row.loss.name,
            "metrics": metrics
        })
        df = df.append(row, ignore_index=True)

Pool: SUM_POOL_4
Opt: Adam
Loss: L1LOSS
DT: RECT_COUNT

Pool: SUM_POOL_4
Opt: Adam
Loss: MSELOSS
DT: RECT_COUNT

Pool: SUM_POOL_4
Opt: SGD
Loss: L1LOSS
DT: RECT_COUNT

Pool: SUM_POOL_4
Opt: SGD
Loss: MSELOSS
DT: RECT_COUNT

Pool: MAX_POOL_4
Opt: Adam
Loss: L1LOSS
DT: RECT_COUNT

Pool: MAX_POOL_4
Opt: Adam
Loss: MSELOSS
DT: RECT_COUNT

Pool: MAX_POOL_4
Opt: SGD
Loss: L1LOSS
DT: RECT_COUNT

Pool: MAX_POOL_4
Opt: SGD
Loss: MSELOSS
DT: RECT_COUNT



KeyboardInterrupt: 

In [18]:
#df = normalize_train_test_df(df, indexes)
df.to_csv('FULL_RESULTS.csv')